# Multi-View-Majority-Vote-Learning-Algorithms-Direct-Minimization-of-PAC-Bayesian-Bounds

This Notebook contains everything necessary to reproduce the experiments in our paper:  

*Multi-View Majority Vote Learning Algorithms: Direct Minimization of PAC-Bayesian Bounds*

## Imports

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict

# Scikit-learn
from sklearn import preprocessing
from sklearn.utils import check_random_state
RAND = check_random_state(42)

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


from mvpb.dNDF import MultiViewBoundsDeepNeuralDecisionForests


# Import data
from mvlearn.datasets import load_UCImultifeature
from data.datasets import (SampleData,
                           MultipleFeatures,
                           Nutrimouse,
                           train_test_split,
                           train_test_merge,
                           s1_s2_split)
from mvpb.util import uniform_distribution

## Load and prepare the multiview datasets

In [2]:
dataset = MultipleFeatures()
X_train, y_train, X_test, y_test = dataset.get_data()
np.unique(y_train)
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
Xs_train = []
Xs_test = []
for xtr, xts in zip(X_train, X_test):
    scaler = preprocessing.MinMaxScaler().fit(xtr)
    Xs_train.append(scaler.transform(xtr))
    Xs_test.append(scaler.transform(xts))

X_train_concat = np.concatenate(Xs_train, axis=1)
X_test_concat = np.concatenate(Xs_test, axis=1)

In [4]:
mvb = MultiViewBoundsDeepNeuralDecisionForests(nb_estimators=3, nb_views=len(Xs_train), depth =3,used_feature_rate=0.8, epochs=5)

In [5]:
mvb.fit(Xs_train,y_train)

View 1/6 done!
View 2/6 done!
View 3/6 done!
View 4/6 done!
View 5/6 done!
View 6/6 done!
returning


MultiViewBoundsDeepNeuralDecisionForests(depth=3, epochs=5, nb_estimators=3,
                                         nb_views=6,
                                         posterior_rho=tensor([0.3333, 0.3333, 0.3333]),
                                         used_feature_rate=0.8)

In [6]:
posterior_Qv , posterior_rho = mvb.optimize_rho('Lambda')

bound=tensor(1.5306, dtype=torch.float64, grad_fn=<AddBackward0>)
upd=tensor(1.5306, dtype=torch.float64, grad_fn=<AddBackward0>)
Difference: 1.999999943436137e-09
Iteration: 0,	 Loss: 0.9093108656438411
Iteration: 1,	 Loss: 0.909262325029647
Iteration: 2,	 Loss: 0.909170139066067
Iteration: 3,	 Loss: 0.9090386536849036
Iteration: 4,	 Loss: 0.9088715246607446
Iteration: 5,	 Loss: 0.9086724091757028
Iteration: 6,	 Loss: 0.9084442361304002
Iteration: 7,	 Loss: 0.9081898003167779
Iteration: 8,	 Loss: 0.9079116111861261
Iteration: 9,	 Loss: 0.907611891613894
Iteration: 10,	 Loss: 0.907292524749749
Iteration: 11,	 Loss: 0.9069553721232593
Iteration: 12,	 Loss: 0.9066020657569792
Iteration: 13,	 Loss: 0.9062339712212061
Iteration: 14,	 Loss: 0.9058526063843101
Iteration: 15,	 Loss: 0.9054587494593499
Iteration: 16,	 Loss: 0.905053669104499
Iteration: 17,	 Loss: 0.9046383242557919
Iteration: 18,	 Loss: 0.9042133900741942
Iteration: 19,	 Loss: 0.903779754886543
Iteration: 20,	 Loss: 0.90333808

In [9]:
_, mv_risk = mvb.predict_MV(Xs_test,y_test)
mv_risk

0.41500000000000004

In [8]:
print(posterior_Qv[0])
print(posterior_Qv[1])
print(posterior_Qv[2])
print(posterior_Qv[3])
print(posterior_Qv[4])
print(posterior_Qv[5])

Parameter containing:
tensor([0.3761, 0.3374, 0.2865], requires_grad=True)
Parameter containing:
tensor([0.3865, 0.2850, 0.3285], requires_grad=True)
Parameter containing:
tensor([0.3285, 0.3675, 0.3040], requires_grad=True)
Parameter containing:
tensor([0.3288, 0.3928, 0.2785], requires_grad=True)
Parameter containing:
tensor([0.3379, 0.3340, 0.3281], requires_grad=True)
Parameter containing:
tensor([0.3468, 0.2983, 0.3548], requires_grad=True)


In [9]:
posterior_rho

Parameter containing:
tensor([0.0882, 0.0540, 0.0956, 0.4107, 0.1910, 0.1605], requires_grad=True)